In [5]:
!pip install contextily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 29.9 MB/s eta 0:00:00


In [6]:
# This is for google collab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as PathEffects

warnings.filterwarnings('ignore', category=FutureWarning, message='.*observed=False.*')

In [8]:
pollution_data = '/content/drive/My Drive/Visualisation in data science/data/madrid_combined.csv'

df_pollution = pd.read_csv(pollution_data)
print(df_pollution.head(1))

                  date  BEN    CO  EBE  MXY  NMHC       NO_2        NOx  OXY  \
0  2001-08-01 01:00:00  NaN  0.37  NaN  NaN   NaN  58.400002  87.150002  NaN   

         O_3   PM10  PXY  SO_2  TCH  TOL   station  year  PM25  NO  CH4  
0  34.529999  105.0  NaN  6.34  NaN  NaN  28079001  2001   NaN NaN  NaN  


In [9]:
stations_data = '/content/drive/My Drive/Visualisation in data science/data/stations.csv'

df_stations = pd.read_csv(stations_data)
print(df_stations.head(1))

         id            name          address       lon        lat  elevation
0  28079004  Pza. de España  Plaza de España -3.712247  40.423853        635


In [61]:
# Data preparation
import pandas as pd
import numpy as np

# Identify all available pollutants in the dataset (excluding non-pollutant columns)
non_pollutant_cols = ['date', 'station', 'year']
all_pollutants = [col for col in df_pollution.columns if col not in non_pollutant_cols]

# Default pollutant to visualize (can be changed)
selected_pollutant = 'NO_2'  # Default pollutant

# Check if selected_pollutant is available, otherwise use the first available pollutant
if selected_pollutant not in all_pollutants and all_pollutants:
    selected_pollutant = all_pollutants[0]

# Filter data for years of interest
years_available = sorted(df_pollution['year'].unique())
# Select first and last year if 2008 and 2018 are not available
start_year = 2008 if 2008 in years_available else years_available[0]
end_year = 2018 if 2018 in years_available else years_available[-1]

df_start = df_pollution[df_pollution['year'] == start_year]
df_end = df_pollution[df_pollution['year'] == end_year]

# Group by station and calculate mean for start year
avg_start = df_start.groupby('station')[all_pollutants].mean().reset_index()

# Group by station and calculate mean for end year
avg_end = df_end.groupby('station')[all_pollutants].mean().reset_index()

# Merge the two dataframes
pollution_change = pd.merge(avg_start, avg_end, on='station', suffixes=(f'_{start_year}', f'_{end_year}'))

# Calculate changes for all pollutants
for pollutant in all_pollutants:
    pollution_change[f'{pollutant}_change'] = pollution_change[f'{pollutant}_{end_year}'] - pollution_change[f'{pollutant}_{start_year}']

# Calculate air quality change based on selected pollutant
# Note: For some pollutants, increasing values may be good (e.g., O_3 at certain levels)
# Default assumption: decrease is good (negative change value is positive for air quality)
pollutant_improves_when_decreasing = {
    'NO_2': True, 'NOx': True, 'PM10': True, 'SO_2': True,
    'CO': True, 'BEN': True, 'TCH': True, 'NO': True, 'PM25': True
}

# Set default behavior for unlisted pollutants
for pollutant in all_pollutants:
    if pollutant not in pollutant_improves_when_decreasing:
        pollutant_improves_when_decreasing[pollutant] = True

# Merge with station data to get coordinates
pollution_map_data = pd.merge(pollution_change, df_stations, left_on='station', right_on='id')

# Prepare yearly data for trend charts for all pollutants
stations_list = pollution_map_data['station'].unique()
# Get yearly data for each station and pollutant
yearly_data = {}

for station in stations_list:
    # Filter data for this station
    station_data = df_pollution[df_pollution['station'] == station]
    if not station_data.empty:
        # Create a dictionary to hold data for each pollutant
        yearly_data[station] = {}
        for pollutant in all_pollutants:
            # Check if this pollutant has data for this station
            if pollutant in station_data.columns and not station_data[pollutant].isna().all():
                # Group by year and calculate average
                yearly_avg = station_data.groupby('year')[pollutant].mean().reset_index()
                yearly_data[station][pollutant] = yearly_avg

In [62]:
# Visualization with ipywidgets dropdown and output clearing
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import contextily as ctx
from matplotlib.patches import Rectangle
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create an output widget to contain the plot
output_widget = widgets.Output()

# Function to create and display the plot
def create_plot(selected_pollutant):
    with output_widget:
        # Clear previous output
        clear_output(wait=True)

        # Create figure with appropriate size
        fig = plt.figure(figsize=(14, 12))
        ax = plt.subplot(111)

        # Create custom colormap for air quality change (red to white to green)
        cmap = LinearSegmentedColormap.from_list('air_quality', ['#d62728', '#ffffff', '#2ca02c'])

        # Set title
        ax.set_title(f'Madrid Air Pollution Change ({start_year}-{end_year}): {selected_pollutant}', fontsize=16)

        # Calculate air quality change based on selected pollutant
        multiplier = -1 if pollutant_improves_when_decreasing.get(selected_pollutant, True) else 1
        pollution_map_data['air_quality_change'] = multiplier * pollution_map_data[f'{selected_pollutant}_change']

        # Normalize the change values for color mapping
        if pollution_map_data['air_quality_change'].notna().sum() > 1:
            valid_changes = pollution_map_data.loc[pollution_map_data['air_quality_change'].notna(), 'air_quality_change']
            if len(valid_changes) > 0:
                vmax = max(abs(valid_changes.max()), abs(valid_changes.min()))
                vmin = -vmax
            else:
                vmin, vmax = -1, 1
        else:
            vmin, vmax = -1, 1

        # Plot stations and line charts
        for idx, row in pollution_map_data.iterrows():
            # Check if we have change data for this station
            if pd.isna(row['air_quality_change']):
                # Use gray for stations with missing data
                color = 'gray'
                size = 100  # Use a small fixed size

                # Plot station
                ax.scatter(row['lon'], row['lat'], s=size, c=color,
                           edgecolor='black', linewidth=1, alpha=0.8, zorder=10)

                # Add "N/A" label
                ax.text(row['lon'], row['lat'] - 0.005, "N/A",
                        fontsize=8, ha='center', va='top',
                        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1),
                        zorder=11)
            else:
                # Determine size based on absolute change (with minimum size)
                size = 150 * (abs(row['air_quality_change']) + 0.5)  # Add 0.5 to ensure minimum size

                # Determine color based on direction of change
                normalized_value = (row['air_quality_change'] - vmin) / (vmax - vmin)
                color = cmap(normalized_value)

                # Plot station
                ax.scatter(row['lon'], row['lat'], s=size, c=[color],
                           edgecolor='black', linewidth=1, alpha=0.8, zorder=10)

            # Add station name
            ax.text(row['lon'], row['lat'] + 0.01, row['name'],
                    fontsize=8, ha='center', va='bottom',
                    bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1),
                    zorder=11)

            # Add line chart next to the station if yearly data is available for this pollutant
            has_data = (row['station'] in yearly_data and
                        selected_pollutant in yearly_data[row['station']] and
                        not yearly_data[row['station']][selected_pollutant].empty)

            if has_data:
                # Get data for this station and pollutant
                station_yearly_data = yearly_data[row['station']][selected_pollutant]

                # Create a small line chart
                # Position the chart offset from the station
                chart_width, chart_height = 0.05, 0.03  # Width and height in coordinate units
                chart_x = row['lon'] + 0.01  # Offset to the right
                chart_y = row['lat'] - 0.02  # Offset below

                # Create a white background for the chart
                background = Rectangle((chart_x, chart_y), chart_width, chart_height,
                                      facecolor='white', alpha=0.8, edgecolor='gray', zorder=8)
                ax.add_patch(background)

                # Get x and y data (years and pollutant values)
                years = station_yearly_data['year'].values
                pollutant_values = station_yearly_data[selected_pollutant].values

                # Normalize years to fit in chart width
                if len(years) > 1:
                    x_norm = chart_x + chart_width * (years - years.min()) / (years.max() - years.min())

                    # Normalize pollutant values to fit in chart height
                    if pollutant_values.max() > pollutant_values.min():
                        y_norm = chart_y + chart_height * (pollutant_values - pollutant_values.min()) / (pollutant_values.max() - pollutant_values.min())

                        # Plot the line chart
                        ax.plot(x_norm, y_norm, 'b-', linewidth=2, zorder=9)

                        # Add dots for the start and end years
                        start_idx = np.where(years == years.min())[0][0]
                        end_idx = np.where(years == years.max())[0][0]

                        ax.plot(x_norm[start_idx], y_norm[start_idx], 'bo', markersize=4, zorder=9)
                        ax.plot(x_norm[end_idx], y_norm[end_idx], 'ro', markersize=4, zorder=9)
            else:
                # If no data available for this pollutant, show "N/A" for the chart
                chart_x = row['lon'] + 0.03  # Offset to the right
                chart_y = row['lat'] - 0.01  # Offset below

                ax.text(chart_x, chart_y, "N/A",
                        fontsize=8, ha='center', va='center',
                        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1),
                        zorder=9)

        # Set axis limits for Madrid area (or use the actual data bounds with some padding)
        lon_min, lon_max = pollution_map_data['lon'].min() - 0.05, pollution_map_data['lon'].max() + 0.05
        lat_min, lat_max = pollution_map_data['lat'].min() - 0.05, pollution_map_data['lat'].max() + 0.05
        ax.set_xlim(lon_min, lon_max)
        ax.set_ylim(lat_min, lat_max)

        # Add the basemap from contextily - this adds the actual map of Madrid
        try:
            # Add basemap (using OpenStreetMap)
            ctx.add_basemap(ax, crs='EPSG:4326', source=ctx.providers.OpenStreetMap.Mapnik)
        except Exception as e:
            print(f"Could not add basemap: {e}")
            # If adding basemap fails, create a simple background
            ax.set_facecolor('#f0f0f0')
            ax.grid(True, linestyle='--', alpha=0.7)

        # Add legend
        handles = [
            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#d62728', markersize=10,
                      label=f'Worsened {selected_pollutant}'),
            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#2ca02c', markersize=10,
                      label=f'Improved {selected_pollutant}'),
            plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', markersize=8,
                      label='Small Change/No Data'),
            plt.Line2D([0], [0], color='blue', linewidth=2, label=f'{selected_pollutant} Trend'),
            plt.Line2D([0], [0], marker='o', color='blue', markersize=5, label=f'{start_year} Value'),
            plt.Line2D([0], [0], marker='o', color='red', markersize=5, label=f'{end_year} Value')
        ]

        ax.legend(handles=handles, loc='lower left', title='Legend', fontsize=10)

        # Remove axis labels for cleaner map view
        ax.set_xticks([])
        ax.set_yticks([])

        plt.tight_layout()
        plt.show()

# Create ipywidgets dropdown
dropdown = widgets.Dropdown(
    options=all_pollutants,
    value=selected_pollutant,
    description='Pollutant:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Function to handle dropdown change
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        create_plot(change['new'])

# Register the callback
dropdown.observe(on_change)

# Display widgets in a vertical layout
display(widgets.VBox([dropdown, output_widget]))

# Create the initial plot
create_plot(selected_pollutant)